In [1]:
using SymPy
using LinearAlgebra
# 
using Parameters

In [2]:
import PyCall
PyCall.pyimport_conda("sympy.physics.quantum.spin", "sympy")
import_from(sympy.physics.quantum.spin, (:WignerD, :wignerd), typ=:Any)
PyCall.pyimport_conda("sympy.physics.wigner",       "sympy")
import_from(sympy.physics.wigner)

In [3]:
Wignerd(J,λ1,λ2,θ) = 
    WignerD(J,λ1,λ2,0,θ,0).doit()
clgn(two_j1,two_m1,two_j2,two_m2,two_j,two_m) = 
    clebsch_gordan(Sym(two_j1)/2, Sym(two_j2)/2, Sym(two_j)/2, Sym(two_m1)/2, Sym(two_m2)/2, Sym(two_m)/2)

clgn (generic function with 1 method)

## Projected $\cos\theta$ distribution

In [4]:
θ, = @vars θ real=true

(θ,)

In [5]:
v = simplify.([sum(abs2, Wignerd(1,λ,ξ,θ) for ξ in [-1, 1]) for λ = [-1, 0, 1]])

3-element Array{Sym,1}:
 cos(θ)^2/2 + 1/2
         sin(θ)^2
 cos(θ)^2/2 + 1/2

In [6]:
let
    H = [Sym("b") Sym("a") Sym("c");
         Sym("a") Sym("d") Sym("a");
         Sym("c") Sym("a") Sym("b")]
    simplify(sum(H[i,j]^2*v[i] for i in 1:3, j=1:3))
end

   2    2         2    2    2       2    2    2       2    2    2       2
- a *cos (θ) + 3*a  + b *cos (θ) + b  + c *cos (θ) + c  - d *cos (θ) + d 

# Part 2: angular distributions

In [7]:
function A4μ(vars; H)
    @unpack θ1,ϕ,θ2 = vars
    return simplify(9/Sym(4)*sum(
        (λ1-λ2 == λ1′-λ2′ ? 1 : 0) *
            Wignerd(1,λ1 ,ξ1,θ1) * Wignerd(1,λ2 ,ξ2,θ2) * (isodd(1-λ2) ? -1 : 1) *
            Wignerd(1,λ1′,ξ1,θ1) * Wignerd(1,λ2′,ξ2,θ2) * (isodd(1-λ2′) ? -1 : 1) *
            (cos((λ1-λ1′)*ϕ)+1im*sin((λ1-λ1′)*ϕ)) *
                 H[λ1+2,λ2+2] *
            conj(H[λ1′+2,λ2′+2])
            for ξ1 in -1:2:1, ξ2 in -1:2:1,
                λ1  in -1:1, λ2  in -1:1,
                λ1′ in -1:1, λ2′ in -1:1))
end

A4μ (generic function with 1 method)

In [8]:
θ1, θ2, ϕ = @vars θ_1 θ_2 ϕ positive=true
vars = (θ1=θ1,ϕ=ϕ,θ2=θ2)

(θ1 = θ_1, ϕ = ϕ, θ2 = θ_2)

In [9]:
a,b,c,d,P,s = @vars a b c d P s real=true

(a, b, c, d, P, s)

In [12]:
Hg = [  b   a     c;
      s*a   d P*s*a;
      s*c P*a   P*b];

In [13]:
A4μ_full = A4μ(vars; H=Hg)

     2  2  2    2         2           2  2  2    2           2  2    2        
  9*P *a *s *sin (θ_1)*sin (θ_2)   9*P *a *s *sin (θ_1)   9*P *a *sin (θ_1)*si
- ------------------------------ + -------------------- - --------------------
                8                           4                          8      

 2           2  2    2           2  2    2         2           2  2    2      
n (θ_2)   9*P *a *sin (θ_2)   9*P *b *sin (θ_1)*sin (θ_2)   9*P *b *sin (θ_1) 
------- + ----------------- + --------------------------- - ----------------- 
                  4                        16                       8         

     2  2    2           2  2        2                                        
  9*P *b *sin (θ_2)   9*P *b    9*P*a *s*sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*c
- ----------------- + ------- + ----------------------------------------------
          8              4                               2                    

             2    2         2         2          

In [16]:
angular_basis = [
    sin(θ1)^2*sin(θ2)^2*sin(ϕ)^2,
    sin(θ1)sin(θ2)cos(θ1)cos(θ2)*cos(ϕ),
    sin(θ1)^2*sin(θ2)^2,
    sin(θ1)^2,
    sin(θ2)^2  
]
# 
norms = [2/Sym(3)*2/Sym(3)*1/Sym(2), 0, 2/Sym(3)*2/Sym(3), 2/Sym(3), 2/Sym(3), 1]
print_norms = map(x->x==0 ? 1 : x, norms)
angular_basis

5-element Array{Sym,1}:
             sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2
 sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)
                      sin(θ_1)^2*sin(θ_2)^2
                                 sin(θ_1)^2
                                 sin(θ_2)^2

In [17]:
function expand_on_basis(ampl, basis)
    lexp = expand(ampl.subs(cos(θ1)^2,1-sin(θ1)^2).subs(cos(θ2)^2,1-sin(θ2)^2).subs(cos(2ϕ),1-2*sin(ϕ)^2))
    lexp = expand(lexp.subs(P^2, 1).subs(s^2, 1))
    coeff = Vector{Sym}(undef,length(basis)+1)
    for (i,b) in enumerate(basis)
        c = collect(lexp, b).coeff(b, 1) 
        coeff[i] = c
        lexp -= c*b
        lexp = expand(lexp)
    end
    coeff[6] = simplify(lexp)
    return simplify.(coeff)
end
#     println(simplify(coeff_K' * [angular_basis...,1] - A4K_full))

expand_on_basis (generic function with 1 method)

In [18]:
coeff_μ = expand_on_basis(A4μ_full, angular_basis)

6-element Array{Sym,1}:
                                        -9*P*b^2/4
                 9*P*a^2*s/2 - 9*P*b*d/4 - 9*b*d/4
 9*P*b^2/8 - 9*a^2/2 + 9*b^2/8 + 9*c^2/8 + 9*d^2/4
                       9*a^2/2 - 9*b^2/4 - 9*c^2/4
                       9*a^2/2 - 9*b^2/4 - 9*c^2/4
                                 9*b^2/2 + 9*c^2/2

In [130]:
simplify(sum(coeff_μ .* norms))

   2      2      2    2
4*a  + 2*b  + 2*c  + d 

In [131]:
print_coeff_μ = coeff_μ .* print_norms

6-element Array{Sym,1}:
                              -P*b^2/2
     9*P*a^2*s/2 - 9*P*b*d/4 - 9*b*d/4
 P*b^2/2 - 2*a^2 + b^2/2 + c^2/2 + d^2
             3*a^2 - 3*b^2/2 - 3*c^2/2
             3*a^2 - 3*b^2/2 - 3*c^2/2
                     9*b^2/2 + 9*c^2/2

# Forgotten particle-2 phase

In [ ]:
Hg′= [  -b   a     -c;
      -s*a   d -P*s*a;
      -s*c P*a   -P*b];
A4μ_full′ = A4μ(vars; H=Hg′);

In [20]:
A4μ_full - A4μ_full′

     2                                                9*P*b*d*sin(θ_1)*sin(θ_2
9*P*a *s*sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ) - ------------------------
                                                                              

)*cos(θ_1)*cos(θ_2)*cos(ϕ)   9*b*d*sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)
-------------------------- - ------------------------------------------------
2                                                   2                        

In [21]:
coeff_μ′ = expand_on_basis(A4μ_full′, angular_basis)
coeff_μ-coeff_μ′

6-element Array{Sym,1}:
                               0
 9*P*a^2*s - 9*P*b*d/2 - 9*b*d/2
                               0
                               0
                               0
                               0

# Ambiguity

In [41]:
mG = A4μ(vars;
    H=[ 0 1 0
        s 0 s*η
        0 η 0])

 2  2    2         2         2  2    2         2    2         2         2    2
s *η *sin (θ_1)*cos (θ_2)   s *η *sin (θ_1)   s *sin (θ_1)*cos (θ_2)   s *sin 
------------------------- + --------------- + ---------------------- + -------
            2                      2                    2                   2 

                                                                              
(θ_1)   s*η*(cos(-2*θ_1 + 2*θ_2 + ϕ) + cos(2*θ_1 - 2*θ_2 + ϕ) - cos(2*θ_1 + 2*
----- + ----------------------------------------------------------------------
                                                           8                  

                                      2    2         2         2    2         
θ_2 - ϕ) - cos(2*θ_1 + 2*θ_2 + ϕ))   η *sin (θ_2)*cos (θ_1)   η *sin (θ_2)   s
---------------------------------- + ---------------------- + ------------ + -
                                               2                   2          

  2         2           2     
in (θ_2)*cos (θ_1)

# 4K analysis

In [79]:
function A4K(vars; H=error("give H"))
    @unpack θ1,ϕ,θ2 = vars
    return expand(9*sum(
        (λ1-λ2 == λ1′-λ2′ ? 1 : 0) *
            Wignerd(1,λ1 ,0,θ1) * Wignerd(1,λ2 ,0,θ2) * (isodd(1-λ2) ? -1 : 1) *
            Wignerd(1,λ1′,0,θ1) * Wignerd(1,λ2′,0,θ2) * (isodd(1-λ2′) ? -1 : 1) *
            (cos((λ1-λ1′)*ϕ)+1im*sin((λ1-λ1′)*ϕ)) *
                 H[λ1+2,λ2+2] *
            conj(H[λ1′+2,λ2′+2])
            for λ1  in -1:1, λ2  in -1:1,
                λ1′ in -1:1, λ2′ in -1:1))
end

A4K (generic function with 2 methods)

In [80]:
A4K_full = A4K(vars; H=Hg)

   2  2  2    2         2           2  2    2         2           2  2    2   
9*P *a *s *sin (θ_2)*cos (θ_1)   9*P *a *sin (θ_1)*cos (θ_2)   9*P *b *sin (θ_
------------------------------ + --------------------------- + ---------------
              2                               2                             4 

      2           2  2    2         2                                         
1)*sin (θ_2)   9*P *c *sin (θ_1)*sin (θ_2)         2                          
------------ + --------------------------- + 18*P*a *s*sin(θ_1)*sin(θ_2)*cos(θ
                            4                                                 

                           2    2         2                                   
                      9*P*b *sin (θ_1)*sin (θ_2)*cos(2*ϕ)                     
_1)*cos(θ_2)*cos(ϕ) + ----------------------------------- - 9*P*b*d*sin(θ_1)*s
                                       2                                      

                                      2  2    2  

In [133]:
coeff_K = expand_on_basis(A4K_full, angular_basis)

6-element Array{Sym,1}:
                                       -9*P*b^2
                   18*P*a^2*s - 9*P*b*d - 9*b*d
 9*P*b^2/2 - 18*a^2 + 9*b^2/2 + 9*c^2/2 + 9*d^2
                                  9*a^2 - 9*d^2
                                  9*a^2 - 9*d^2
                                          9*d^2

In [134]:
simplify(sum(coeff_K .* norms))

   2      2      2    2
4*a  + 2*b  + 2*c  + d 

In [135]:
print_coeff_K = coeff_K .* print_norms

6-element Array{Sym,1}:
                                -2*P*b^2
            18*P*a^2*s - 9*P*b*d - 9*b*d
 2*P*b^2 - 8*a^2 + 2*b^2 + 2*c^2 + 4*d^2
                           6*a^2 - 6*d^2
                           6*a^2 - 6*d^2
                                   9*d^2

## Print nice latex form

In [129]:
sympy.print_latex.(map(x->x.subs(θ1,Sym("theta1")).subs(θ2,Sym("theta2")).subs(ϕ,Sym("phi")), angular_basis))

\sin^{2}{\left (\phi \right )} \sin^{2}{\left (\theta_{1} \right )} \sin^{2}{\left (\theta_{2} \right )}
\sin{\left (\theta_{1} \right )} \sin{\left (\theta_{2} \right )} \cos{\left (\phi \right )} \cos{\left (\theta_{1} \right )} \cos{\left (\theta_{2} \right )}
\sin^{2}{\left (\theta_{1} \right )} \sin^{2}{\left (\theta_{2} \right )}
\sin^{2}{\left (\theta_{1} \right )}
\sin^{2}{\left (\theta_{2} \right )}


5-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing

In [132]:
sympy.print_latex.(print_coeff_μ)

- \frac{P b^{2}}{2}
\frac{9 P a^{2} s}{2} - \frac{9 P b d}{4} - \frac{9 b d}{4}
\frac{P b^{2}}{2} - 2 a^{2} + \frac{b^{2}}{2} + \frac{c^{2}}{2} + d^{2}
3 a^{2} - \frac{3 b^{2}}{2} - \frac{3 c^{2}}{2}
3 a^{2} - \frac{3 b^{2}}{2} - \frac{3 c^{2}}{2}
\frac{9 b^{2}}{2} + \frac{9 c^{2}}{2}


6-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [136]:
sympy.print_latex.(print_coeff_K)

- 2 P b^{2}
18 P a^{2} s - 9 P b d - 9 b d
2 P b^{2} - 8 a^{2} + 2 b^{2} + 2 c^{2} + 4 d^{2}
6 a^{2} - 6 d^{2}
6 a^{2} - 6 d^{2}
9 d^{2}


6-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [137]:
sympy.print_latex(Hg)

\left[\begin{matrix}b & a & c\\a s & d & P a s\\P c & P a & P b\end{matrix}\right]
